In [1]:
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

In [2]:
df = yf.download(tickers='ZOMATO.NS', period='1y', interval='1h')

# df2 = yf.download(tickers='ZOMATO.NS', period='475d', interval='1d')

# df3 = yf.download(tickers='TCS.NS', period='475d', interval='1d')

# df4 = yf.download(tickers='HDFCBANK.NS', period='475d', interval='1d')

[*********************100%***********************]  1 of 1 completed


In [3]:
# df1.to_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\API_R.csv")

In [4]:
# df3

In [5]:
# df4.shape

In [6]:
# df = pd.concat([df1,df2,df3,df4],ignore_index = True)

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-22 09:15:00+05:30,61.700001,62.000000,60.049999,60.950001,60.950001,0
2022-12-22 10:15:00+05:30,61.000000,61.150002,59.750000,60.250000,60.250000,5684077
2022-12-22 11:15:00+05:30,60.200001,60.599998,59.950001,60.200001,60.200001,4318543
2022-12-22 12:15:00+05:30,60.200001,61.200001,60.099998,61.000000,61.000000,5407990
2022-12-22 13:15:00+05:30,61.000000,61.450001,59.900002,60.049999,60.049999,9484416
...,...,...,...,...,...,...
2023-12-21 11:15:00+05:30,126.000000,126.750000,125.599998,126.550003,126.550003,5687090
2023-12-21 12:15:00+05:30,126.599998,127.500000,126.400002,126.900002,126.900002,5127050
2023-12-21 13:15:00+05:30,126.900002,127.000000,125.199997,125.349998,125.349998,5430433


In [8]:
# df['Date'] = pd.to_datetime(df['Date'])
# df = df.set_index('Date')

data = df.filter(["Open", "High", "Low", "Close",])
dataset = data.values

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [10]:
seq_len = 60
train_size = int(len(scaled_data) * 0.6)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - seq_len:]

In [11]:
x_train, y_train = [], []
for i in range(seq_len, len(train_data)):
    x_train.append(train_data[i - seq_len: i])
    y_train.append(train_data[i])

x_train, y_train = np.array(x_train), np.array(y_train)

In [12]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences = False))
model.add(Dropout(0.3))
# model.add(LSTM(64, return_sequences = False))
# model.add(Dropout(0.4))
# model.add(LSTM(64))
# model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(4))

In [13]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=15, epochs=10)

Epoch 1/10


: 

In [ ]:
model.save(r"C:\Users\VikashKumarChaudhary\Documents\LSTM/Model_V5.h5")

In [ ]:
# Load the test data
from keras.models import load_model
from sklearn.metrics import mean_squared_error

test_data = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\Stock_Final\RELIANCE2.csv")
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data = test_data.set_index('Date')
test_dataset = test_data.filter(["Open", "High", "Low", "Close"]).values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_test_data = scaler.fit_transform(test_dataset)

seq_len = 60
x_test, y_test = [], []
for i in range(seq_len, len(scaled_test_data)):
    x_test.append(scaled_test_data[i - seq_len: i])
    y_test.append(scaled_test_data[i])

x_test, y_test = np.array(x_test), np.array(y_test)

model = load_model(r"C:\Users\VikashKumarChaudhary\Documents\LSTM/Model_V5.h5")

predictions = model.predict(x_test)

predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test, predictions)

accuracy = np.sqrt(mse)

print("RMSE:", accuracy)


37/37 [==============================] - 1s 15ms/step
RMSE: 132.44942328241396
